# 📥📤Inputs and Outputs (Model I/O)

In LangChain, Model I/O (Input/Output) is the layer that defines how you interact with models. It provides standardized abstractions for sending prompts into models and parsing their responses.

## 🔑 Why do we need Model I/O
* **Consistency across models**: You can swap between OpenAI, Anthropic, Llama, etc. without changing your business logic, since LangChain standardizes the I/O.
* **Structured outputs**: Instead of working with free-form text, you can parse model responses into JSON, Pydantic objects, or enums. This reduces errors and hallucinations.
* **Composable pipelines**: Model I/O integrates with other LangChain pieces (retrievers, memory, tools) so you can **chain models and functions easily**.
* **Flexibility**:
    * Fine control over prompts (system + human messages, variable substitution).
    * Choice of parsers (regex, structured, function-calling).
    * Input/output transformations before/after the model.

# 📝 Prompt Templates
A Prompt template in LangChain is a reusable way to create dynamic prompts for LLMs.
Instead of hardcoding the full prompt string, you define a template with placeholders, and later fill those placeholders with variables at runtime.
This ensures:

* Consistency – Reuse the same structure for different inputs.
* Flexibility – Easily swap variables without rewriting prompts.
* Maintainability – Keeps your LLM app organized as prompts grow.

LangChain Uses two types of prompt templates
1. **PromptTemplate** (for completion models)
2. **ChatPromptTemplate** (for chat models)

## ✨PromptTemplate
* **Purpose**: Used for text-based LLMs (completion models).
* **Output**: A single string prompt.
* **Use case**: When you want to generate a plain text prompt (no roles like “system” or “user”).

In [ ]:
## google colab
# %pip install langchain openai
# %pip install langchain langchain-community openai
# %pip install langchain_openai

In [17]:
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import OpenAI

In [18]:
load_dotenv()
# os.environ["OPENAI_API_KEY"] = "<Your API KEY>" # Google colab

In [19]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0, api_key=os.getenv("OPEN_API_KEY"))

In [20]:
from langchain.prompts import PromptTemplate

# Single input variable: "product"
single_input_template = PromptTemplate(
    input_variables=["product"],
    template="What are three benefits of using {product}?"
)

# template = "What are three benefits of using {product}?"
# prompt = PromptTemplate.from_template(template)
# print(prompt.format(product="LangChain"))

# Format with input
formatted_prompt = single_input_template.format(product="LangChain")
print(formatted_prompt)

What are three benefits of using LangChain?


In [22]:
response = llm.invoke(formatted_prompt)
print("LLM response:", response)

LLM response: 

1. Increased Efficiency: LangChain utilizes blockchain technology to automate and streamline language translation processes, resulting in increased efficiency and reduced turnaround times. This can save businesses time and resources, allowing them to focus on other important tasks.

2. Enhanced Security: With its decentralized and immutable nature, LangChain offers enhanced security for language translation services. This means that translations are less susceptible to tampering or manipulation, ensuring the accuracy and integrity of the translated content.

3. Cost Savings: By eliminating intermediaries and automating processes, LangChain can significantly reduce the costs associated with language translation services. This makes it a more affordable option for businesses and individuals looking to translate content into multiple languages.


## 💬ChatPromptTemplate
* **Purpose**: Used for chat-based LLMs (like OpenAI’s GPT-4, GPT-4o, GPT-3.5).
* **Output**: A list of message objects (with roles: system, human, ai, etc.).
* **Use case**: When you need structured multi-role prompts (system instructions + user queries).

In [ ]:
# Document as a string
document_text = """
LangChain is a framework for developing applications powered by language models.
It provides abstractions for prompts, chains, agents, memory, and integrates with many external tools.
"""

# Define the chat prompt template with system + human messages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the provided document to answer questions."),
    ("human", "Here is the document:\n{document}\n\nQuestion: {question}")
])

# Format the prompt with input variables
messages = chat_prompt.format_messages(
    document=document_text,
    question="What does LangChain provide?"
)

# Initialize the model
chat = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.getenv("OPEN_API_KEY"))

# Get the response
response = chat.invoke(messages)
print(response.content)


In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [ ]:
# Document string
document_text = """
LangChain is a framework for developing applications powered by language models.
It provides abstractions for prompts, chains, agents, memory, and integrates with many external tools.
"""

# Create system and human message prompt templates
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant. Use the provided document to answer questions."
)

human_prompt = HumanMessagePromptTemplate.from_template(
    "Here is the document:\n{document}\n\nQuestion: {question}"
)

# Build the chat prompt template
chat_prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])

# Format with inputs
messages = chat_prompt.format_messages(
    document=document_text,
    question="What does LangChain provide?"
)

# Initialize LLM
chat = ChatOpenAI(model="gpt-4o-mini", temperature=0,  api_key=os.getenv("OPEN_API_KEY"))

# Invoke the model
response = chat.invoke(messages)
print(response.content)

# Few Shots Prompt-Template

In LangChain, a few-shot template is a way to guide a language model by giving it examples of how it should respond before asking the real question. The idea comes from few-shot learning, where instead of training the model with lots of data, you show it just a handful of demonstrations right inside the prompt.

LangChain makes this easier with **FewShotPromptTemplate**. Instead of manually writing out all examples in one long string, you define a template for the examples and then let LangChain format them dynamically.

In [33]:
from langchain.prompts import (
    FewShotPromptTemplate, 
    FewShotChatMessagePromptTemplate, 
    PromptTemplate, 
    ChatPromptTemplate, 
    HumanMessagePromptTemplate, 
    AIMessagePromptTemplate
)

from langchain_openai import ChatOpenAI

In [34]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=os.getenv("OPEN_API_KEY"))

In [35]:
# Template for each example
example_prompt = PromptTemplate(
    input_variables=["review", "sentiment"],
    template="Review: {review}\nSentiment: {sentiment}\n"
)

# Few-shot examples
examples = [
    {"review": "I loved the storyline and the acting was brilliant!", "sentiment": "Positive"},
    {"review": "The movie was too long and really boring.", "sentiment": "Negative"},
    {"review": "Amazing visuals and great soundtrack.", "sentiment": "Positive"},
]

# FewShotPromptTemplate with suffix for new input
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Review: {input}\nSentiment:",
    input_variables=["input"]
)

# Format with a new review
print(few_shot_prompt.format(input="The plot was confusing and poorly executed."))
# 4. Instantiate an LLM


# 5. Format the prompt with a new review
formatted_prompt = few_shot_prompt.format(input="The plot was confusing and poorly executed.")
print("=== Final Prompt Sent to LLM ===")
print(formatted_prompt)

# 6. Call the LLM
response = llm.invoke(formatted_prompt)
print("\n=== Model Output ===")
print(response.content)


Review: I loved the storyline and the acting was brilliant!
Sentiment: Positive


Review: The movie was too long and really boring.
Sentiment: Negative


Review: Amazing visuals and great soundtrack.
Sentiment: Positive


Review: The plot was confusing and poorly executed.
Sentiment:
=== Final Prompt Sent to LLM ===
Review: I loved the storyline and the acting was brilliant!
Sentiment: Positive


Review: The movie was too long and really boring.
Sentiment: Negative


Review: Amazing visuals and great soundtrack.
Sentiment: Positive


Review: The plot was confusing and poorly executed.
Sentiment:

=== Model Output ===
Negative


In [40]:
# 1) Build a chat prompt for each example (human → ai)
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Review: {review}"),
    ("ai", "{sentiment}"),
])

# 2) Few-shot examples
chat_examples = [
    {"review": "I loved the storyline and the acting was brilliant!", "sentiment": "Positive"},
    {"review": "The movie was too long and really boring.", "sentiment": "Negative"},
    {"review": "Amazing visuals and great soundtrack.", "sentiment": "Positive"},
]

# 3) Few-shot block (no input_variables here)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=chat_examples,
    example_prompt=example_prompt,
)

# 4) Full chat prompt (system + few-shot + new human input)
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that classifies movie reviews as Positive or Negative."),
    few_shot_prompt,
    ("human", "Review: {input}"),
])

# 6) Format and invoke
messages = chat_prompt.format_messages(input="The plot was confusing and poorly executed.")
print("=== Final Prompt Messages ===")
for m in messages:
    print(m.type.upper() + ":", m.content)

resp = llm.invoke(messages)
print("\n=== Model Output ===")
print(resp.content)


=== Final Prompt Messages ===
SYSTEM: You are a helpful assistant that classifies movie reviews as Positive or Negative.
HUMAN: Review: I loved the storyline and the acting was brilliant!
AI: Positive
HUMAN: Review: The movie was too long and really boring.
AI: Negative
HUMAN: Review: Amazing visuals and great soundtrack.
AI: Positive
HUMAN: Review: The plot was confusing and poorly executed.

=== Model Output ===
Negative
